In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Legacy Feature Store to Feature Store 2.0 Migration**

---
## **Introduction**


Vertex AI Feature Store is a fully managed feature management solution provided by Google Cloud Platform. It allows organizations to efficiently serve, share, and reuse machine learning (ML) features across projects and teams. As part of Google Cloud's commitment to enhancing its services, a new version of Feature Store (2.0) has been introduced with improved capabilities and performance.


This notebook guides you through the migration process from Vertex AI Legacy Feature Store to the new Feature Store 2.0. We'll cover configuration setup, offline store migration, and online store migration.

Let's get started!


---

## **Configuration Setup**

The migration process is driven by a configuration file (config.yaml) that defines the feature stores and entity types to be migrated.

1. **Create `config.yaml`:**
   - Create a YAML file named `config.yaml` in your project directory.
   - Fill in the following details:
   
  ```yaml
  project_id: "your-gcp-project-id"  # Replace with your project ID
  region: "your-region"  # Replace with your region (e.g., "us-central1")
  
  bq_dataset_prefix: exp_db_
  bq_table_prefix: exp_db_tbl_

  legacy_feature_store:
    feature_store_mode: all  # Or "list" to specify individual stores
    # feature_stores:  # Only if feature_store_mode is "list"
  ```


- **Explanation:**
   - `project_id`: This is your Google Cloud Project ID where the feature stores are located.
   - `region`: The region where your feature stores reside.
   - `bq_dataset_prefix`: Prefix for the BigQuery dataset name
   - `bq_table_prefix`: Prefix for the BigQuery table name
   - `legacy_feature_store`:
     - `feature_store_mode`:  
       - `all`: Migrates all feature stores in your project.
       - `list`: Migrates specific feature stores listed in `feature_stores`.


- **Feature Store Mode: "list" Example:**

  ```yaml
  project_id: your-project-id  # Replace with your project ID
  region: your-region  # Replace with your region (e.g., "us-central1")

  legacy_feature_store:
    feature_store_mode: list  # Or "all"
    feature_stores:  # Only if feature_store_mode is "list"
      - name: legacy_feature_store_name
        entity_type_mode: all  # "all" or "list"
        entity_types:
          - name: entity_type_1
            entity_id_column: entity_id_new
            mapping_file: path/to/mapping_file_1.csv
          - name: entity_type_2
            entity_id_column: entity_id_new
            mapping_file: path/to/mapping_file_2.csv
  ```

- **Explanation of Feature Store Mode: "list":**
  * **`project_id`**: Your Google Cloud Project ID.
  * **`region`**: The Google Cloud region where your feature stores are located.
  * **`legacy_feature_store`**: Configuration for the legacy Vertex AI Feature Store.
      * **`feature_store_mode`**:
          * **`list`**: Process only the feature stores specified in the `feature_stores` section.
      * **`feature_stores`**: A list of feature store configurations.
          * **`name`**: The name of the legacy feature store to migrate.
          * **`entity_type_mode`**:
              * **`all`**: Process all entity types in the feature store.
              * **`list`**: Process only the entity types specified in the `entity_types` section.
          * **`entity_types`**: A list of entity type configurations.
              * **`name`**: The name of the entity type within the feature store.
              * **`entity_id_column`**: The name of the entity id column to rename.
              * **`mapping_file`**: The path to the CSV file containing the feature mapping for this entity type.

2. **Create Feature Mapping Files (Optional):**
   - Create CSV files for each entity type you want to rename features for.
   - Each row should have `original_feature_name` and `destination_feature_name`.
   - Example:

  ```csv
  original_feature_name,destination_feature_name
  id,id
  name,name_new
  style,style_new
  ```


---
### **Set Root Dir**

In [ ]:
import os
import sys

# Add the project root directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

---
## **Offline Store Migration**

### **Setting Up Logging**

In [ ]:
from logging_config import configure_logging

configure_logging()

These lines are responsible for preparing your project to automatically log important information as it runs. Here's what they do:

1. **Import the Configuration:**
   - The line `from logging_config import configure_logging` brings in a special function named `configure_logging` from a separate file called `logging_config.py`. This function is where all the details about how you want your logs to be handled are defined.

2. **Activate the Logging Setup:**
   - `configure_logging()` is then executed. This function sets up Loguru, a powerful logging library, to do several things:
      - **Create a "logs" folder:** It makes sure a special folder called "logs" exists within your project directory. This is where all the log files will be stored.
      - **Set up log files:**
         - It creates a main log file named `app.log` where a complete history of your program's events will be kept.
         - It also creates a separate log file for each time you run your program. This filename includes a timestamp so you can easily track which log corresponds to which run.
      - **Define log levels:**  It determines what kind of information gets logged.  It sets the default level to `DEBUG`, which means you'll see all kinds of messages, including detailed technical ones. It also sets up a special log level called `REQUEST_TIME` that you can use to track how long specific tasks take.

In essence, these two lines prepare your project to keep a record of its activities. By calling `configure_logging()`, you ensure that every time you run your program, a new log file is created to store important information about what happened during that run, while the main `app.log` file keeps a running record for you to review later.


### **Export Legacy Stores**

In [ ]:
from legacy_exporter import LegacyExporter

legacy_exporter = LegacyExporter()
export_response = legacy_exporter.export_feature_store()

**The `LegacyExporter` Class: The Conductor of the Migration**

The `LegacyExporter` class is the heart of the offline store migration process. Its primary responsibility is to manage the extraction of data from the Legacy Feature Store and its subsequent preparation for import into Feature Store 2.0.

* **Initialization:**
    - When you create a `LegacyExporter` object, it reads the `config.yaml` file to understand your specific migration settings.
    - It establishes a connection to the Feature Store service in your Google Cloud project using the project ID and region you provided in the configuration.

* **`export_feature_store()` Method:**
    - This is the core method that does the heavy lifting. Here's what it does, step by step:
        1. **Gather Feature Store Information:** It identifies which feature stores and entity types to migrate based on the configuration file.
        2. **Create BigQuery Destinations:** For each feature store and entity type, it creates dedicated datasets and tables in BigQuery to temporarily hold the exported data.
        3. **Extract Feature Values:** It fetches the feature values (the actual data points) from the Legacy Feature Store.
        4. **Apply Feature Mapping:** If you've provided a feature mapping file, it renames the features according to the mapping before exporting.
        5. **Export to BigQuery:** It writes the extracted and potentially renamed feature values to the appropriate BigQuery tables.
        6. **Organize Results:** It prepares a dictionary (`export_response`) that summarizes what was exported (which feature stores, entity types, features, and the location of the data in BigQuery).


### **Import into Feature Store 2.0**

In [ ]:
from feature_store_importer import FeatureStoreImporter

feature_store_importer = FeatureStoreImporter()
feature_store_importer.import_features(data=export_response)

**The `FeatureStoreImporter` Class: The Architect of Feature Store 2.0**

The `FeatureStoreImporter` class acts as the primary tool for constructing your new Feature Store 2.0 structure. It takes the organized data from BigQuery (prepared by the `LegacyExporter`) and transforms it into Feature Groups and Features, the building blocks of Feature Store 2.0.

* **Initialization:**
    - When a `FeatureStoreImporter` object is created, it, like its counterpart `LegacyExporter`, reads the `config.yaml` file to understand the project settings.
    - It uses two essential clients:
        - **FeaturestoreServiceClient:** For general interactions with the Feature Store.
        - **FeatureRegistryServiceClient:** Specifically for creating and managing Feature Groups and Features.

* **`import_features(data)` Method:**
    - This method orchestrates the entire import process. Here's a simplified breakdown:

        1. **Check for Data:** If the input `data` (the `export_response` from `LegacyExporter`) is empty, it logs a message and exits gracefully.

        2. **Iterate Over Exported Data:** It iterates over the data for each feature store and entity type.

        3. **Create Feature Groups:** For each entity type, it creates a new Feature Group in Feature Store 2.0 using the entity type's name. This Feature Group will act as a container for the related features.

        4. **Create Features:** Within each Feature Group, it iterates over the list of features and creates individual Feature entries. These Features represent the specific data points you're storing.

        5. **Associate with BigQuery:** It associates each Feature Group with the corresponding BigQuery table where the data is stored, allowing seamless access for future use.


---
## **Online Store Migration**

### **Generate Intermediate Online FS serving config file**

In [ ]:
import json
from utils import transform_json

ONLINE_STORE_CONFIG_FILE = os.path.join(project_root, 'config', 'online_store_config.json')

# Generate Intermediate Online FS serving config file
transformed_config = transform_json(export_response)
with open(ONLINE_STORE_CONFIG_FILE, "w", encoding="utf-8") as f:
    json.dump(transformed_config, f)

The code snippet above acts as a bridge between the offline store migration (where features and entities are exported) and the online store creation in Feature Store 2.0.

- **Configuration Transformation:**
It takes the raw output (export_response) from the offline feature store export process and transforms it into a format that's compatible with the creation of a FeatureOnlineStore in Feature Store 2.0. This transformation involves mapping legacy feature store configurations to the new Feature Store 2.0 schema.

- **Intermediate Configuration File (ONLINE_STORE_CONFIG_FILE):** The transformed configuration is written to a JSON file named online_store_config.json (specified by ONLINE_STORE_CONFIG_FILE). This file serves as a crucial intermediate step in the migration process.

- **Customization Point:** Since Feature Store 2.0 has evolved from the legacy system, there might be new configurations or differences in how settings are applied. This file provides an opportunity for users to fine-tune the configuration to match their specific needs and preferences.


### **Review the Online FS serving config file**

**Points to note while reviewing the online store config**

1. The `online_store_type` can be set to `bigtable` or `optimized` depending on the use case and the size of features.
2. Currently, the script supports Manual or Cron as Sync modes for FeatureViews. The default value for the field `cron_schedule` will be set to `null` which will mark it as a Manual Sync Mode.
3. The fields `bigtable_min_node_count`,`bigtable_max_node_count`,`cpu_utilization_target` will be skipped if the `online_store_type` is set to `optimized`.
4. Review the mapping of Feature Groups to Feature views.

### **Read back the Online FS serving config file**

In [ ]:
def read_json_config(config_file):
    """Reads a JSON file and returns its contents as a dictionary"""
    with open(config_file, 'r', encoding="utf-8") as f:
        data = json.load(f)
    return data


# Read Online Store config File
online_store_config = read_json_config(ONLINE_STORE_CONFIG_FILE)

### **Create Online Stores & Feature Views**

In [ ]:
from logging_config import logger
from online_store_creator import FeatureOnlineStore


for online_store_config_obj in online_store_config["online_stores"]:
    online_store_obj = FeatureOnlineStore(online_store_config_obj=online_store_config_obj,
                                          project_id=online_store_config["project_id"],
                                          region=online_store_config["region"])
    try:
        online_store_obj.create_feature_online_store()
    except ValueError as e:
        logger.error(f"Error creating online store: {e}")
        continue
    online_store_obj.create_feature_views_from_feature_groups()



**Online Store Creation and Feature View Population:**

This code automates the setup of online feature stores and their associated feature views in Vertex AI Feature Store 2.0, using configurations specified in the `online_store_config.json` file. Here's a detailed breakdown of what happens:

1. **Iterate Over Online Store Configurations:**
   - The code loops through each entry in the `online_stores` list within the `online_store_config.json` file. Each entry represents the desired configuration for a single online feature store.

2. **Instantiate FeatureOnlineStore Object:**
   - For each configuration, a `FeatureOnlineStore` object is created. This object encapsulates the settings and actions needed to create and manage the online store.
   - The object is initialized with:
      - `online_store_config_obj`: The specific configuration dictionary for the current online store, including `online_store_type`, `bigtable_min_node_count`, `bigtable_max_node_count`, and `cpu_utilization_target`.
      - `project_id`: The Google Cloud project ID where the online store will be created.
      - `region`: The region where the online store will be hosted.

3. **Create Online Store:**
   - The `create_feature_online_store()` method is called on the `FeatureOnlineStore` object. This method performs the following:
      - **Check for Existence:** It first checks if an online store with the same name (`feature_online_store_name` from the configuration) already exists. If so, it logs an error and skips to the next configuration.
      - **Create Online Store Request:** If the store doesn't exist, it constructs a request to create a new online store, using the following details:
          - **Feature Online Store ID:** The unique name specified in the configuration (`feature_online_store_name`).
          - **Configuration:**
              - If `online_store_type` is "bigtable", it includes settings like `bigtable_min_node_count`, `bigtable_max_node_count`, and `cpu_utilization_target`.
              - If `online_store_type` is "optimized", it sets up an optimized store.

      - **Send Request and Poll:** It sends the request and waits (polls) for the operation to complete. If successful, a new online store is created with the specified name and configuration. If there's an error during creation, it's logged, and the process moves on.

4. **Create Feature Views:**
   - After successfully creating the online store, the `create_feature_views_from_feature_groups()` method is called. This method:
      - **Iterate Over Feature Views:**  It iterates over the `feature_views` list within the online store configuration. Each entry represents a feature view to be created.
      - **Feature Group Mapping:** For each feature view, it identifies the associated feature groups and feature IDs from the configuration.
      - **Feature View Creation Request:** It constructs a request to create a new feature view in the online store, associating it with the specified feature groups and IDs. It also sets the refresh schedule based on the `cron_schedule` field in the configuration (or defaults to a Manual sync if not specified).
      - **Send Request and Poll:** It sends the request and polls the operation until it completes. If successful, a new feature view is created in the online store, providing access to the features from the associated feature groups.


---
## **Validation**

In [ ]:
!python3 validation.py --project_id [PROJECT_ID] --region [REGION_ID] --spreadsheet_file_path [PATH_FOR_OUTPUT_SPREADSHEET]

- **Explanation:**
   - Runs the `validation.py` script to compare the results in Legacy Feature Store and Feature Store 2.0.
   - Make sure to replace placeholders with your actual project ID, region, and spreadsheet path.
   - Output will be stored in the specified spreadsheet, showing feature stores, entities, and features from both versions.